In [1]:
import os

# Why Torch? You'll find the answer in the .md files! 
os.environ["KERAS_BACKEND"] = "torch"

import keras
from models.simple_cnn.simple_cnn import SimpleCNNContainer
import keras_tuner
from k_fold_cv.k_fold_generator import KFoldController
from utils.dataset_loader import dataset_loader

from torch.utils.data import DataLoader
## Let's try Grayscale
hyperparameters = keras_tuner.HyperParameters()
train, test = dataset_loader((224, 224), is_grayscale=False)

# todo change name of KFoldController and also check it is shuffled
# I am using the KFoldController class to simply get 20% of the dataset as a validation fold. I won't do any
# k-Fold CV on the parameters search or else this will take forever
dataset_split_controller = KFoldController(5)
dataset_split_controller.load_data(train)

local_train, validation = dataset_split_controller.get_data_for_fold(0)

train_dataloader = DataLoader(dataset=local_train, batch_size=32)
validation_dataloader = DataLoader(dataset=validation, batch_size=32)
hypermodel = SimpleCNNContainer((3, 224, 224))
# We are learning the structure of the network with fixed parameters for learning.
# As we know hyperparamters of NN are: Neurons, Layers, Optimizer, Learning-Rate, Epochs, Batch size etc
hyperparameters.Fixed("lr", value=1e-3)


## TODO VEDI QUESTI PARAMETERI https://keras.io/api/keras_tuner/tuners/base_tuner/#tuner-class
tuner = keras_tuner.Hyperband(
    hypermodel,
    hyperparameters=hyperparameters,
    hyperband_iterations=3,
    objective='val_loss',
    tune_new_entries=True,
    overwrite=True,
    directory="naive_dnn_search",
    project_name="grayscale_fixed_lr" # should be highest possible
)

tuner.search_space_summary()
tuner.search(train_dataloader, epochs=30, validation_data=validation_dataloader,
             callbacks=[
                 # keras.callbacks.TensorBoard("/tmp/tb_logs"),
                 keras.callbacks.EarlyStopping(monitor='val_loss', patience=4)
             ])

Trial 31 Complete [00h 00m 46s]

Best val_loss So Far: 0.5786980390548706
Total elapsed time: 00h 45m 17s

Search: Running Trial #32

Value             |Best Value So Far |Hyperparameter
0.001             |0.001             |lr
4                 |4                 |kernel_0
64                |64                |filters_0
False             |False             |second_conv_layer
3                 |3                 |kernel_1
48                |48                |filters_1
96                |32                |hidden_units
2                 |2                 |tuner/epochs
0                 |0                 |tuner/initial_epoch
4                 |4                 |tuner/bracket
0                 |0                 |tuner/round
Epoch 1/2
119/119 ━━━━━━━━━━━━━━━━━━━━ 0s 316ms/step - accuracy: 0.5576 - loss: 0.6815

Traceback (most recent call last):
  File "/home/jacopo/miniconda3/envs/keras-pytorch/lib/python3.12/site-packages/keras_tuner/src/engine/base_tuner.py", line 274, in _try_run_and_update_trial
    self._run_and_update_trial(trial, *fit_args, **fit_kwargs)
  File "/home/jacopo/miniconda3/envs/keras-pytorch/lib/python3.12/site-packages/keras_tuner/src/engine/base_tuner.py", line 239, in _run_and_update_trial
    results = self.run_trial(trial, *fit_args, **fit_kwargs)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/jacopo/miniconda3/envs/keras-pytorch/lib/python3.12/site-packages/keras_tuner/src/tuners/hyperband.py", line 427, in run_trial
    return super().run_trial(trial, *fit_args, **fit_kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/jacopo/miniconda3/envs/keras-pytorch/lib/python3.12/site-packages/keras_tuner/src/engine/tuner.py", line 314, in run_trial
    obj_value = self._build_and_fit_model(trial, *args, **copied_kwa

RuntimeError: Number of consecutive failures exceeded the limit of 3.
Traceback (most recent call last):
  File "/home/jacopo/miniconda3/envs/keras-pytorch/lib/python3.12/site-packages/keras_tuner/src/engine/base_tuner.py", line 274, in _try_run_and_update_trial
    self._run_and_update_trial(trial, *fit_args, **fit_kwargs)
  File "/home/jacopo/miniconda3/envs/keras-pytorch/lib/python3.12/site-packages/keras_tuner/src/engine/base_tuner.py", line 239, in _run_and_update_trial
    results = self.run_trial(trial, *fit_args, **fit_kwargs)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/jacopo/miniconda3/envs/keras-pytorch/lib/python3.12/site-packages/keras_tuner/src/tuners/hyperband.py", line 427, in run_trial
    return super().run_trial(trial, *fit_args, **fit_kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/jacopo/miniconda3/envs/keras-pytorch/lib/python3.12/site-packages/keras_tuner/src/engine/tuner.py", line 314, in run_tria

In [ ]:
local_train.cumsum(dim=0)

In [ ]:
len(local_train)

In [2]:
tuner.get_best_hyperparameters()